# `Hit the Gym: MiniProject for RL Games CPP`

The exercises in this notebook are meant for CVI aspirants who wish to work on the RL Games CPP. 

**About the project:** The goal of the RL Games project is to explore and analyse current Reinforcement Learning Methods by deploying them on Atari games, Minesweeper, slither.io etc. Further, the techniques developed can be used to attack more complex problems such as Reconnaissance Blind Chess and Autonomous Driving.

In this notebook, you'll use a popular Reinforcement Learning Technique called Q-learning to train an agent to play simple games using the python library OpenAI Gym. 

You may refer to the following while coding:

Python reference: https://bit.ly/3ajUalZ and https://bit.ly/2UgiZKa

OpenAI Gym Documentation: https://gym.openai.com/docs/


### `RL Basics:`
Reinforcement learning is an approach used in Machine Learning where the agent is allowed to interact with the system to learn its behaviour and come up with an optimal startegy to achieve an objective. The agent models the problem as a probabilistic state machine (a graph where the transition from one node to another has a probability distribution). Nodes in the model graph are called states and a transition from one state to another is called an action. Each state transition (which is a (state, action) pair) has a corresponding reward or penalty. The goal of the RL agent is to maximise the reward. 

Training an RL agent from scratch requires us to model the state space and the action space. In addition, we must also come up with suitable rewards for each state transition. The RL agent estimates this reward structure and executes actions so as to maximise them. The final performance of the RL agent is heavily dependent on how the system is modelled. Luckily for us, we **do not need to get into the mathematics** of Reinforcement Learning right now, thanks to the Python library Gym.


Gym offers many in-built RL environments which you can use to play around with. These environments are Python classes with their state spaces, action spaces and rewards pre-defined. You will use two such environments (Taxi-v3 and FrozenLake8x8-v0) to train an agent accomplish a goal. You can find the documentation for these environments here:

Taxi-v3 Documentation: https://gym.openai.com/envs/Taxi-v3/  

FrozenLake Documentation: https://gym.openai.com/envs/FrozenLake8x8-v0/  

To create a gym environment of 'Taxi-v3' you do this:

In [1]:
import gym
import numpy as np  
import random

# Create an environment of Taxi-v3:
env = gym.make('Taxi-v3').env 
env.render()
print(env.s)
print(env.observation_space)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

226
Discrete(500)


env.s is the current state of the environment.

env.observation_space() returns the type and size of the observation or state space. Discrete implies that the states are discrete and not continuous. For games like Pacman, Gym uses another type of state space- Box- due to the large number of states. 

**Solving the Problem:** Now the goal of this game is to train the taxi to efficiently pick the passenger from the blue spot and drop them at the purple spot. The taxi can do the following actions: Move Up, Move Down, Move to the Left, Move to the Right, Pickup, Dropoff. The reward structure is such:
1. -10 points for illegal dropoff/pickup actions
2. +20 points if the passenger is dropped off at the correct location
3. -1 point for every other action

A paleolithic approach to this problem would be to pick an action at random and execute it. Eventually the passenger would get picked up and then dropped off at the correct location.

In [6]:
state = env.reset()
epochs = 0
penalty, reward = 0, 0  # Penalty records the number of times the agent hits a wall
frames = []
done = False # Find out the role of 'done' and complete the statement for its initial condition 
while not done: #insert condition:
    '''
      Enter your code here
      The code must pick an action from the action space at random, execute it and update 'penalty' accordingly
    '''
    action = env.action_space.sample()
    state, reward, done, misc = env.step(action)
    
    if reward == -10:
        penalty += 1
    
    frames.append({'frame': env.render(mode='ansi'), 'state': state, 'action': action, 'reward': reward})
    epochs += 1
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalty))

Timesteps taken: 263
Penalties incurred: 66


Run the following cell to visualise the performance of the agent. It won't come as a surprise to see that the approach is quite bad. It is so because, the agent has no memory of the past and hence learns nothing. 

In [7]:
from IPython.display import clear_output
from time import sleep
def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait = True)
        print(frame['frame'])
        print(f"Timestep: {i+1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(0.1)
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 263
State: 85
Action: 5
Reward: 20


## `Enter Q-learning`

A popular technique used in Reinforcement Learning is to have the agent maintain an estimate of the rewards that it would gain from executing a particular state transition (called the Q-table) which is updated after each step based on the reward obtained. The agent picks the action that yields the maximum reward at that particular state. That is, if a particular state transition ((state, action) pair) resulted in a good reward, the agent is better off repeating that action whenever that state is attained. 

**TASK 1:**

Find out the update rule for Q-learning and implement Q-learning for the Taxi-v3 problem. Also, find out what 'exploration versus exploitation' is and use a suitable way to optimise on exploration and exploitation.

You would need to set a few hyperparameters- learning rate ($\alpha$), reward decay rate ($\gamma$), number of episodes and exploration probability($\epsilon$). Obtain the performance characteristics of the agent (that is, number of epochs per episode and average number of penalties per episode) for ($\alpha$, $\gamma$, episodes, $\epsilon$) = (0.6, 0.9, 1000, 0.4)

In [10]:
'''
  Q-LEARNING ON Taxi-v3:
  
  Exploration vs exploitation refers to the two choices an RL agent has when prompted to make a move
  in its learning phase. Exploration means to execute different actions toget a better idea of the 
  possible rewards at a particular state. Exploitation means to use the current knowledge of rewards,
  instead of exploring new actions, to choose the best move possible at the current state.
  
  Both are necessary as an agent which only explores (i.e, random choice all the time) will not always be taking
  the best move at a state. And an agent which only exploits without exploring will know only one path which
  most likely is not optimal, as new actions may lead to better rewards.
  '''

q_table = np.zeros([env.observation_space.n, env.action_space.n])

# Hyperparameters
alpha = 1.0
gamma = 0.9
epsilon = 0.4

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        # The update rule of Q-learning
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value
        
        # q_table[state, action] += alpha * (reward + gamma*next_max - old_value)
        
        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.



In [12]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 1000

for i in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)
        if i == 1:
            print(state)
        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Average epochs per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

381
281
261
241
221
121
21
1
17
117
217
237
257
157
57
77
97
85
Average epochs per episode: 13.042
Average penalties per episode: 0.0


**TASK 2:**

Now that you have trained the agent on Taxi-v3, try Q-learning on the FrozenLake8x8-v0 environment. After training, obtain the following performance characteristics- number of epochs per episode and average number of wins. What can you do to improve the performance of the agent?

In [3]:
'''
    Displaying the Frozen Lake env.
'''

env = gym.make('FrozenLake8x8-v0').env 
env.render()
print(env.s)
print(env.observation_space)


SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
0
Discrete(64)


In [24]:
'''
  Q-LEARNING ON FrozenLake8x8-v0:
  I first tried with the same parameters as above for alpha gamma and epsilon.
  However, after some tweaking, I found that a higher gamma gave better performance.
  
  This is possibly because for each episode the game layout does not change, and with
  many episodes all the state-action pairs will be explored. Gamma is the discount factor, which decides
  whether tp give more importance to immediate rewards or later rewards. 
  Once enough exploration is done, if we give enough importance to future rewards, the table will 'learn'
  the optimal path from the start to end.
  
  With gamma=1, I was able to get 100% win rate (see below next cell). 
  The number of moves taken are very large though.
  '''

q_table = np.zeros([env.observation_space.n, env.action_space.n])

# Hyperparameters
alpha = 0.3
gamma = 1.0
epsilon = 0.5

# For plotting metrics
all_epochs = []
all_penalties = []



for i in range(1, 10001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        if done == True and reward == 0:
            reward = -10
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == 0:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 10000
Training finished.



In [29]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties, total_wins = 0, 0, 0
episodes = 1000

for i in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if done == True and reward == 0:
            penalties += 1
        if done == True and reward == 1:
            total_wins += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Average epochs per episode: {total_epochs / episodes}")
#print(f"Average penalties per episode: {total_penalties / episodes}")
print(f"Wins: {total_wins}")

Average epochs per episode: 116.342
Wins: 1000


In [28]:
state = env.reset()
epochs, penalties, reward = 0, 0, 0
frames = []
done = False

while not done:
    
    action = np.argmax(q_table[state])
    state, reward, done, info = env.step(action)
    
    if reward == 0:
        penalties += 1
    frames.append({'frame': env.render(mode='ansi'), 'state': state, 'action': action, 'reward': reward})
    epochs += 1
    if epochs > 1000:
        break
    
print_frames(frames)

  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG

Timestep: 178
State: 63
Action: 2
Reward: 1.0


In [27]:
# Ignore, Diagnostic Print
print(q_table[53])

[-7.56607133 -7.8801563  -7.19745258 -8.21138163]


In [17]:
q_table = np.zeros([env.observation_space.n, env.action_space.n])

# Hyperparameters
alpha = 0.3
gamma = 1.0
epsilon = 0.5
hp_c = 1
best_ratio = 0
best_a = -1
best_g = -1
best_e = -1
best_q_table = np.zeros([env.observation_space.n, env.action_space.n])
for a in [val * 0.1 for val in range(11)]:
    for g in [val * 0.1 for val in range(11)]:
        for e in [val * 0.1 for val in range(11)]:
            q_table = np.zeros([env.observation_space.n, env.action_space.n])
            
            #train
            for i in range(1, 1001):
                state = env.reset()

                epochs, penalties, reward, = 0, 0, 0
                done = False
    
                while not done:
                    if random.uniform(0, 1) < e:
                        action = env.action_space.sample() # Explore action space
                    else:
                        action = np.argmax(q_table[state]) # Exploit learned values

                    next_state, reward, done, info = env.step(action)
  
                    old_value = q_table[state, action]
                    next_max = np.max(q_table[next_state])
        
                    if done == True and reward == 0:
                        reward = -10
        
                    new_value = (1 - a) * old_value + a * (reward + g * next_max)
                    q_table[state, action] = new_value

                    if reward == 0:
                        penalties += 1

                    state = next_state
                    epochs += 1
                    if epochs > 1000:
                        break
        
                if i % 100 == 0:
                    clear_output(wait=True)
                    print(hp_c)
                    print(f"Episode: {i}")
            hp_c += 1

            #print("Training finished.\n")
            
            total_epochs, total_penalties, total_wins = 0, 0, 0
            episodes = 1000

            for i in range(episodes):
                state = env.reset()
                epochs, penalties, reward = 0, 0, 0
    
                done = False
    
                while not done:
                    action = np.argmax(q_table[state])
                    state, reward, done, info = env.step(action)

                    if done == True and reward == 0:
                        penalties += 1
                    if done == True and reward == 1:
                        total_wins += 1

                    epochs += 1
                    if epochs>1000:
                        break

                total_penalties += penalties
                total_epochs += epochs
            
            if total_wins > best_wins:
                best_wins = total_wins
                best_q_table = q_table
                best_a, best_g, best_e = a,g,e
                best_ratio = total_wins/(total_epochs/episodes)
            elif total_wins == best_wins and total_wins/(total_epochs/episodes) > best_ratio:
                best_wins = total_wins
                best_q_table = q_table
                best_a, best_g, best_e = a,g,e
                best_ratio = total_wins/(total_epochs/episodes) 
                
print("Best wins out of 1000 : ", best_wins)
print("Best parameters : ", best_a, best_g, best_e)


1331
Episode: 1000
Best wins out of 1000 :  1000
Best parameters :  0.2 0.7000000000000001 0.1


In [18]:
q_table = np.zeros([env.observation_space.n, env.action_space.n])

# Hyperparameters
alpha = best_a
gamma = best_g
epsilon = best_e

# For plotting metrics
all_epochs = []
all_penalties = []



for i in range(1, 1001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        if done == True and reward == 0:
            reward = -10
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == 0:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

total_epochs, total_penalties, total_wins = 0, 0, 0
episodes = 1000

for i in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if done == True and reward == 0:
            penalties += 1
        if done == True and reward == 1:
            total_wins += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Average epochs per episode: {total_epochs / episodes}")
#print(f"Average penalties per episode: {total_penalties / episodes}")
print(f"Wins: {total_wins}")

state = env.reset()
epochs, penalties, reward = 0, 0, 0
frames = []
done = False

while not done:
    
    action = np.argmax(q_table[state])
    state, reward, done, info = env.step(action)
    
    if reward == 0:
        penalties += 1
    frames.append({'frame': env.render(mode='ansi'), 'state': state, 'action': action, 'reward': reward})
    epochs += 1
    if epochs > 1000:
        break
    
print_frames(frames)

Episode: 1000
Training finished.



KeyboardInterrupt: 

## `Further Motivation`

In case you are curious about the mathematics of Reinforcement Learning, you check the following resources out:

RL Lectures by Dr. David Silver: http://www0.cs.ucl.ac.uk/staff/d.silver/web/Teaching.html

Medium Blog Post on RL techniques: https://medium.com/@jonathan_hui/rl-introduction-to-deep-reinforcement-learning-35c25e04c199

Deep Neural Networks (useful for Deep RL): http://cs231n.github.io/

If you are facing any issue or difficulty, you may PM (on WhatsApp):

Arjun: 97392 19819